# Multi-Agent with Tools

In this notebook we present a sample multi-agent architecture, where the agent orchestating the answer calls world knowledge, a tool implemented using another agent that iteratively searches the web. Additionally, a simple tool to get Youtube links based on a query is included, so that the agent's recommendations can include visual data as well.

> If you don't have `pytube` installed, you need to install it inside the notebook.

In [ ]:
# In case pytube is not installed
!pip install pytube

In [ ]:
import os
from gptstonks.multiagents.graphs import GraphAgentWithTools
from gptstonks.multiagents.tools import WorldKnowledgeTool, YoutubeSearchTool
from llama_index.llms.openai import OpenAI as LlamaIndexOpenAI

Your OpenAI key, as function calling is used in all the agents:

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

Define the tools to search the Internet and YouTube

In [ ]:
llamaindex_llm = LlamaIndexOpenAI(model="gpt-3.5-turbo-0125", temperature=0, max_tokens=2048)
world_knowledge_tool = WorldKnowledgeTool.from_llamaindex_llm(
    llamaindex_llm=llamaindex_llm,
    use_openai_agent=True,
    verbose=True,
    tool_description="Useful to retrieve any information related to music and trends: songs, what users like, recent updates, etc.. Input must be a complete question of the information to retrieve.",
    search_tool_description="Useful to search any type of information using DuckDuckGo.",
    wikipedia_tool_description="Useful to get information about named entities.",
    auto_multistep_query_engine_qa_template="Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query. Write very detailed answers.\nQuery: {query_str}\nAnswer: ",
    auto_multistep_query_engine_refine_template="You are an expert Q&A system that strictly operates in two modes when refining existing answers:\n1. **Rewrite** an original answer using the new context.\n2. **Repeat** the original answer if the new context isn't useful.\nNever reference the original answer or context directly in your answer.\nWrite very detailed answers.\nWhen in doubt, just repeat the original answer.\nNew Context: {context_msg}\nQuery: {query_str}\nOriginal Answer: {existing_answer}\nNew Answer: ",
    auto_multistep_query_engine_stepdecompose_query_prompt="The original question is as follows: '{query_str}'\nWe must answer this question from a knowledge source, by dividing it into simpler questions. Context information for the knowledge source is provided below, as well as previous reasoning steps.\nGiven the context and previous reasoning, return a relevant question that can be answered from the context and helps answer the original question. This relevant question can be the same as the original question if it is simple, or this question can represent a step towards answering the overall question. It must be relevant to answer the original question.\nIf we cannot extract more information from the context, provide 'None' as the answer, but NEVER if the previous reasoning is 'None' too.\n\nQuestion: {query_str}\nKnowledge source context: {context_str}\nPrevious reasoning: {prev_reasoning}\nNew question (can't be 'None'): "
)
youtube_search_tool = YoutubeSearchTool.create()

Define the agent graph with OpenAI function calling

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0, max_tokens=2048, model_kwargs={"top_p": 0.8})
graph = GraphAgentWithTools(
    model=llm,
    tools=[world_knowledge_tool, youtube_search_tool],
    prompt_main_agent=ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Forget everything you knew. You are an expert in creating music playlists for a wide variety of users. You provide very detailed and clearly structured answers. You always start by searching with world knowledge.",
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    ),
).define_basic_graph()

We call the multi-agent architecture to solve our complex task!

In [ ]:
res = await graph.ainvoke({
    "input": "I really like the group AJR. What other groups would you recommend? Provide a Youtube links for reference."
})

Print the final response!

In [ ]:
print(res["context_messages"][-1].content)